In [1]:
!pip install -qU transformers==4.46.3 tokenizers==0.20.3 einops addict easydict

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 37.5 MB/s eta 0:00:00


In [2]:
!pip install -qU flash-attn==2.7.3 --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 90.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
!pip install -qU quanto==0.0.11

In [4]:
!pip install -qU optimum-quanto accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.3/165.3 kB 14.1 MB/s eta 0:00:00


In [5]:
import torch, torchvision
print(torch.__version__)
print(torchvision.__version__)


2.8.0+cu126
0.23.0+cu126


In [6]:
from transformers import AutoModel, AutoTokenizer, QuantoConfig
import torch
import os
from quanto import quantize, freeze


quant_config = QuantoConfig(weights="int8")
# model = transformers.AutoModelForCausalLM.from_pretrained(
#     "meta-llama/Llama-3.1-8B",
#     dtype="auto",
#     device_map="auto",
#     quantization_config=quant_config
# )


os.environ["CUDA_VISIBLE_DEVICES"] = '0'
model_name = 'deepseek-ai/DeepSeek-OCR'

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name,
                                  # _attn_implementation='flash_attention_2',
                                  device_map="auto",
                                  trust_remote_code=True,
                                  use_safetensors=True,
                                  quantization_config = quant_config)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

modeling_deepseekocr.py: 0.00B [00:00, ?B/s]

...

conversation.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/deepseek-ai/DeepSeek-OCR:
- conversation.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/deepseek-ai/DeepSeek-OCR:
- modeling_deepseekocr.py
- configuration_deepseek_v2.py
- deepencoder.py
- modeling_deepseekv2.py
- conversation.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
You are using a model of type deepseek_vl_v2 to instantiate a model of type DeepseekOCR. This is not supported for all configurations of models and can yield errors.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-000001.safetensors:   0%|          | 0.00/6.67G [00:00<?, ?B/s]

Some weights of DeepseekOCRForCausalLM were not initialized from the model checkpoint at deepseek-ai/DeepSeek-OCR and are newly initialized: ['model.vision_model.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
print(model)

DeepseekOCRForCausalLM(
  (model): DeepseekOCRModel(
    (embed_tokens): Embedding(129280, 1280)
    (layers): ModuleList(
      (0): DeepseekV2DecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): QLinear(in_features=1280, out_features=1280, bias=False)
          (k_proj): QLinear(in_features=1280, out_features=1280, bias=False)
          (v_proj): QLinear(in_features=1280, out_features=1280, bias=False)
          (o_proj): QLinear(in_features=1280, out_features=1280, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): DeepseekV2MLP(
          (gate_proj): QLinear(in_features=1280, out_features=6848, bias=False)
          (up_proj): QLinear(in_features=1280, out_features=6848, bias=False)
          (down_proj): QLinear(in_features=6848, out_features=1280, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): DeepseekV2RMSNorm()
        (post_attention_layernorm): DeepseekV2RMSNorm()
      )
      (1-11): 11 x Deeps

In [9]:
model1 = model.eval().cuda()

In [10]:
print(model1)

DeepseekOCRForCausalLM(
  (model): DeepseekOCRModel(
    (embed_tokens): Embedding(129280, 1280)
    (layers): ModuleList(
      (0): DeepseekV2DecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): QLinear(in_features=1280, out_features=1280, bias=False)
          (k_proj): QLinear(in_features=1280, out_features=1280, bias=False)
          (v_proj): QLinear(in_features=1280, out_features=1280, bias=False)
          (o_proj): QLinear(in_features=1280, out_features=1280, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): DeepseekV2MLP(
          (gate_proj): QLinear(in_features=1280, out_features=6848, bias=False)
          (up_proj): QLinear(in_features=1280, out_features=6848, bias=False)
          (down_proj): QLinear(in_features=6848, out_features=1280, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): DeepseekV2RMSNorm()
        (post_attention_layernorm): DeepseekV2RMSNorm()
      )
      (1-11): 11 x Deeps

In [11]:
prompt = "<image>\n<|grounding|>Convert the document to markdown. "
image_file = '/content/batch1-0001.jpg'
output_path = '/content/'

res = model1.infer(tokenizer, prompt=prompt, image_file=image_file, output_path = output_path, base_size = 1024, image_size = 640, crop_mode=True, save_results = True, test_compress = True)

/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Cal

BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([6, 100, 1280])
<|ref|>title<|/ref|><|det|>[[78, 190, 155, 204]]<|/det|>
# Seller:

<|ref|>text<|/ref|><|det|>[[82, 216, 315, 228]]<|/det|>
Andrews, Kirby and Valdez

<|ref|>text<|/ref|><|det|>[[82, 232, 280, 243]]<|/det|>
58861 Gonzalez Prairie

<|ref|>text<|/ref|><|det|>[[82, 247, 317, 258]]<|/det|>
Lake Daniellefurt, IN 57228

<|ref|>text<|/ref|><|det|>[[82, 277, 258, 288]]<|/det|>
Tax Id: 945-82-2137

<|ref|>text<|/ref|><|det|>[[82, 292, 381, 303]]<|/det|>
IBAN: GB75MCRL06841367619257

<|ref|>title<|/ref|><|det|>[[78, 325, 153, 338]]<|/det|>
# ITEMS

<|ref|>table<|/ref|><|det|>[[78, 353, 920, 711]]<|/det|>

<table><tr><td>No.</td><td>Description</td><td>Qty</td><td>UM</td><td>Net price</td><td>Net worth</td><td>VAT [%]</td><td>Gross<br>worth</td></tr><tr><td>1.</td><td>CLEARANCE! Fast Dell Desktop<br>Computer PC DUAL CORE<br>WINDOWS 10 4/8/16GB RAM</td><td>3,00</td><td>each</td><td>209,00</td><td>627,00</td><td>10%</td><td>689,

image: 0it [00:00, ?it/s]
other: 100%|██████████| 10/10 [00:00<00:00, 78251.94it/s]
